# Merge and Transform Data

- Load pre-processed data from various CSV files and merge into a single dataframe
- Check for issues such as possible duplicates and incorrect dates, and flag these rows for review
- Transform data so that it is organized into columns consistent with the data entry tool
- Save the cleaned data to CSV file, which will be imported to Google Sheets for manual editing by CCMF

In [1]:
import pandas as pd
import numpy as np

save_csv = True

## Utility Functions

In [2]:
def concatenate_text_cols(df, cols, sep='; '):
    """Concatenate 2 or more text columns and deal with missing values"""
    output = df[cols[0]].copy()
    for col in cols[1:]:
        output = (output.str.cat(df[col], sep=sep, na_rep='')
                  .str.replace('^' + sep, '', regex=True)
                  .str.replace(sep + '$', '', regex=True)
                  .replace('', np.nan)
                 )
    
    return output

def clean_incident_category(s):
    """Utility function for standardizing the incident categories"""
    if pd.isnull(s):
        return np.nan
    
    if s == 'C' or s.startswith('C '):
        category = 'Criminal: Unknown'
    elif s == 'NC' or s.startswith('NC'):
        category = 'Non-Criminal'
    else:
        category = np.nan
    return category

## Load Pre-Processed Data

In [3]:
# Consolidated raw data
rr = pd.read_csv('race_relations_raw_consolidated.csv')
rr['date'] = pd.to_datetime(rr['date'])

# Cleaned incident locations
locations = (pd.read_csv('Locations/incident_locations.csv')
             .drop(['article_url', 'national'], axis=1)
            )  

# Article publication dates scraped from each URL
pub_dates = pd.read_csv('Publication date/publish_dates.csv')
pub_dates['publish_date'] = pd.to_datetime(pub_dates['publish_date'])
cols = ['incident_id', 'article_url']
if not pub_dates[cols].equals(rr[cols]):
    raise ValueError('Incident IDs or URLs inconsistent between publication dates dataframe and main dataframe')

# Merge dataframes
rr = (rr.merge(pub_dates.drop('article_url', axis=1), on='incident_id', how='outer')
      .merge(locations, on='incident_id', how='outer')
     )

# Add detailed locations for the few incidents where it can be easily parsed from the description
detailed_locs = ['Fort McMurray Islamic Centre', 'South West Detention Centre', 'Brock University',
                 'Brantford Mosque', 'University of Manitoba', 'Laurentian University', 'Nipissing University',
                 'University of Saskatchewan', 'University of New Brunswick', 'London Muslim Mosque', 
                 'Eaton Centre', 'York University', "Queen's University", 'Chinese Cultural Centre', 
                 'Canadian Museum for Human Rights', 'McMaster University']
for val in detailed_locs:
    rr.loc[rr['description'].str.contains(val, case=False).fillna(False), 'detailed_location'] = val
    
# Rough fix to remove obvious inaccuracies in publication dates
date1 = pd.to_datetime('2020-01-01')
date2 = pd.to_datetime('2020-08-01')
idx_remove = (rr['publish_date'] > date1) & (rr['date'] < date1)
idx_remove = idx_remove | (rr['publish_date'] > date2) & (rr['date'] < date2)
idx_remove = idx_remove | (rr['publish_date'] < rr['date'].min())
rr.loc[idx_remove, 'publish_date'] = np.nan
print(f'QA/QC: Removed {idx_remove.sum()} likely inaccurate publication dates')

# Consolidate all previous tags into one column
cols = ['category', 'sub_category', 'target_community', 'location']
rr['previous_tags'] = concatenate_text_cols(rr, cols)

# Standardize the incident category column
rr['category'] = rr['category'].apply(clean_incident_category)

# Specify whether there were charges associated with a crime if this info was
# provided in the previous tags, otherwise leave as Criminal: Other
charges = {
    "victim didn’t file a charge": 'Criminal: Not charged',
    'not charged': 'Criminal: Not charged',
    'Charged with mischief' : 'Criminal: Charged',
    'forgery charges' : 'Criminal: Charged'
}
for sub_str, category in charges.items():
    rr.loc[rr['previous_tags'].str.contains(sub_str), 'category'] = category

# Possible duplicate entries
grouped = rr[rr.duplicated(subset=['article_url'], keep=False)].groupby('article_url')
dup_labels = {}
for i, (url, df) in enumerate(grouped, start=1):
    group_ids = df.reset_index()['incident_id'].astype(str).to_list()
    group_str = f'Group {i:02d}: URL duplicated in multiple incidents'
    dup_labels.update({int(n) : group_str for n in group_ids})

rr['dup_notes'] = rr['incident_id'].map(dup_labels)
rr.loc[rr['dup_notes'].notnull(), 'dup_flag'] = 'Possible duplicate incident'

# Check for dates where the year is inconsistent with the sheet name
idx_check_date = rr['year'] != rr['orig_sheet_name'].str[:4].astype(int)
rr.loc[idx_check_date, 'date_flag'] = 'Check date'
rr.loc[idx_check_date, 'date_notes'] = 'Date could not be parsed or listed date is inconsistent with year in original sheet name'

# Check for entries with multiple article URLs
idx_url = rr['article_url'].str.contains(' ') | rr['article_url'].str.contains('\n')
rr.loc[idx_url, 'url_flag'] = 'Invalid Article URL'
rr.loc[idx_url, 'url_notes'] = 'Multiple URLs listed - remove extras so there is only one for this incident'

# Concatenate additional data flags and notes
rr['needs_review'] = concatenate_text_cols(rr, ['dup_flag', 'date_flag', 'url_flag', 'needs_review'])
rr['notes'] = concatenate_text_cols(rr, ['dup_notes', 'date_notes', 'url_notes', 'notes'])

# Drop unneeded columns
drop_cols = ['location', 'sub_category', 'target_community', 'year', 'orig_sheet_name', 'orig_row_num',
             'date_flag', 'date_notes', 'dup_flag', 'url_flag', 'url_notes']
rr = rr.drop(drop_cols, axis=1)

print(rr.shape)
rr.head()

QA/QC: Removed 48 likely inaccurate publication dates
(1000, 13)


,incident_id,date,category,description,article_url,publish_date,city_or_region,province,detailed_location,needs_review,notes,previous_tags,dup_notes
0,1,2018-01-05,Non-Criminal,B.C. woman caught on video delivering racist r...,https://globalnews.ca/news/3949365/b-c-woman-c...,2018-01-05,Burnaby,British Columbia,NaN,NaN,NaN,NC; hate speech; anti-black; Burnaby,NaN
1,2,2018-01-03,Non-Criminal,Windsor Muslims pen letter asking police why I...,https://www.cbc.ca/news/canada/windsor/graffit...,2018-01-03,Windsor,Ontario,NaN,NaN,NaN,NC; vandalism; anti-muslim; Windsor,NaN
2,3,2018-01-04,Non-Criminal,Sen. Lynn Beyak booted from Conservative caucu...,https://nationalpost.com/news/politics/sen-lyn...,2018-01-05,Ottawa,Ontario,NaN,NaN,NaN,NC; hate speech; anti-indigenous; Ottawa,NaN
3,4,2018-01-09,Non-Criminal,Lawyers question fairness of Calgary judge acc...,https://www.cbc.ca/news/canada/calgary/judge-e...,2018-01-09,Calgary,Alberta,NaN,NaN,NaN,NC; hate speech; anti-black; Calgary,NaN
4,5,2018-01-20,Non-Criminal,P.E.I. legion to apologize after video shows w...,http://nationalpost.com/news/canada/p-e-i-legi...,2018-01-20,Tignish,Prince Edward Island,NaN,NaN,NaN,NC; harassment; anti-sikh/indian; Tignish,NaN


## Transform

Map the old tags from the original data into the columns and tags of the new taxonomy.

In [4]:
# Previous tags for each incident
tags_old = (pd.read_csv('Pre-Processing/tags_all.csv')
            .drop('variable', axis=1)
            .drop_duplicates()
            .reset_index(drop=True)
            .rename({'tag' : 'Previous Tag'}, axis=1)
           )
print(tags_old.shape)
tags_old.head()

(2536, 2)


,incident_id,Previous Tag
0,1,anti-black
1,1,hate speech
2,2,anti-muslim
3,2,vandalism
4,3,anti-indigenous


In [5]:
# Lookup table to map old tags to the taxonomy
lookup = pd.read_csv('Pre-Processing/tags_lookup.csv')
print(lookup.shape)
lookup.head()

(110, 13)


,Previous Tag,Ethnic Community,Black/African/Caribbean,Central Asian,East Asian,Indigenous,Latin American,Middle Eastern,South Asian,Southeast Asian,Identity-Based,Context,Type of Incident
0,allegations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,allegations against police,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protective services: Police,NaN
2,anti-afghan,Central Asian,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,anti-african,Black/African/Caribbean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,anti-arab,Middle Eastern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def find_matches(rr, tags_dict, tag_name='Context', drop_duplicates=False, case=False):
    """Find sub-string matches in incident description and assign new tags"""
    extra_tags = pd.DataFrame()
    for tag, pattern in tags_dict.items():
        matches = rr.loc[rr['description'].str.contains(pattern, case=case).fillna(False), 'incident_id']
        extra_tags = extra_tags.append(pd.DataFrame({'incident_id' : matches.values, tag_name : tag}))

    if drop_duplicates:
        extra_tags = extra_tags.drop_duplicates(subset=['incident_id'], keep='first')
        
    return extra_tags


def smush_tags(df, columns, by='incident_id', sep=', ', sort=True):
    """Smush tags on multiple rows into one row of comma-separated tags per incident"""
    
    def smush(vals, sep):
        parts = vals[vals.notnull()].unique()
        if sort:
            parts = sorted(parts)
        output = sep.join(parts)
        if output == '':
            output = np.nan
        return output
    
    df_out = pd.DataFrame()
    for col in columns:
        df_out[col] = df.groupby(by)[col].agg(smush, sep=sep)
        
    df_out = df_out.reset_index(drop=False)
        
    return df_out

In [7]:
# Map old tags to taxonomy
tags = tags_old.merge(lookup, on='Previous Tag', how='left')

# Add other tags based on info in the incident description
online = {
    'Online: Social media' : 'social.media|facebook|twitter|forum|iron march',
    'Online: Other' : 'webinar|website|gofundme|online'
}

edu = {
    'Educational institute: K-12' : 'elementary school|middle school|high school|secondary school',
    'Educational institute: Post-secondary' : 'university|college|law school',
    'Educational institute: Other' : 'school|teacher'
}

other_contexts = {
    'Healthcare/public health' : 'health.?care|health services|hospital',
    'Sports: Hockey' : 'hockey',
    'Sports: Basketball' : 'basketball',
    'Sports: Football' : 'football',
    'Transit' : 'transit|\Wbus\W|^bus\W|\Wbus$|subway',
    'Workplace' : 'worksite|workplace|work environment|at work'
}

types = {
    'Hate speech: Rally/protest' : 'anti-immigration rally|white nationalist rally',
    'Hate speech: Letter/flyer' : 'flyer|anti-semitic letter|anti-arab letter|racist.*letter|letter.*racist|troubling.*letter|controversial letter',
    'Hate speech: Imagery' : 'image'
}

tags = tags.append(find_matches(rr, online, tag_name='Context', drop_duplicates=True))
tags = tags.append(find_matches(rr, edu, tag_name='Context', drop_duplicates=True))
tags = tags.append(find_matches(rr, other_contexts, tag_name='Context'))
tags = tags.append(find_matches(rr, types, tag_name='Type of Incident'))

# Only keep rows which have at least one value in the taxonomy
new_cols = lookup.drop('Previous Tag', axis=1).columns.to_list()
tags = tags[tags[new_cols].any(axis=1)]

# Collapse into one row per incident
tags = smush_tags(tags, columns=new_cols)

# Manually remove extraneous tags for letter/flyer incidents
tags['Type of Incident'] = tags['Type of Incident'].replace({'Hate speech: Letter/flyer, Hate speech: Other' : 'Hate speech: Letter/flyer'})

print(tags.shape)
tags.head()

(904, 13)


,incident_id,Ethnic Community,Black/African/Caribbean,Central Asian,East Asian,Indigenous,Latin American,Middle Eastern,South Asian,Southeast Asian,Identity-Based,Context,Type of Incident
0,1,Black/African/Caribbean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hate speech: Other
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Religious discrimination: Islamophobia,NaN,Vandalism
2,3,Indigenous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Online: Other,Hate speech: Other
3,4,Black/African/Caribbean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hate speech: Other
4,5,South Asian,NaN,NaN,NaN,NaN,NaN,NaN,India,NaN,Religious discrimination: Anti-Sikhism,NaN,Harassment: Other


In [8]:
# Merge with main dataframe and sort so that duplicate URL groups are together and 
# all entries flagged for review are at the top
rr_clean = (rr.drop(new_cols, errors='ignore')
            .merge(tags, on='incident_id', how='outer')
            .sort_values(['needs_review', 'dup_notes', 'incident_id'])
            .drop('dup_notes', axis=1)
            .reset_index(drop=True)
           )

# Rename columns
names_dict = {
    'incident_id' : 'Incident ID',
    'date' : 'Incident Date',
    'category' : 'Incident Category',
    'description' : 'Incident Description',
    'article_url' : 'Article URL',
    'city_or_region' : 'City or Region',
    'province' : 'Province',
    'detailed_location' : 'Detailed Location',
    'needs_review' : 'Needs Review',
    'notes' : 'D4G Notes',
    'previous_tags' : 'Previous Tags',
    'publish_date' : 'Publication Date'
}

rr_clean = rr_clean.rename(names_dict, axis=1)

# Add column with difference between publication date and incident date
rr_clean['Pub Date Minus Incident Date'] = (rr_clean['Publication Date'] - rr_clean['Incident Date']).dt.days

# Fill some of the missing ethnic communities
fill_values = [
    ('Nigeria', 'Nigeria', 'Black/African/Caribbean'),
    ('Somali', 'Somalia', 'Black/African/Caribbean'),
    ('Iran', 'Iran', 'Central Asian'),
    ('Inuit', 'Inuit', 'Indigenous'),
    ('Egypt', 'Egypt', 'Middle Eastern')
]

for pattern, sub_community, community in fill_values:
    idx = rr_clean['Incident Description'].str.contains(pattern).fillna(False)
    rr_clean.loc[idx, 'Ethnic Community'] = rr_clean.loc[idx, 'Ethnic Community'].fillna(community)
    rr_clean.loc[idx, community] = rr_clean.loc[idx, community].fillna(sub_community)

# Add empty columns for manual data entry
cols_extra = ['Gender of Victim(s)', 'Name of Victim(s)', 'CCMF Notes', 'Status']
for col in cols_extra:
    rr_clean[col] = np.nan

# Re-order columns
cols = rr_clean.columns.to_list()
cols_out = ['Incident ID', 'Article URL', 'Incident Description', 'Publication Date',  'Incident Date', 
            'Pub Date Minus Incident Date', 'Province', 'City or Region', 
            'Detailed Location', 'Incident Category', 'Gender of Victim(s)', 'Name of Victim(s)',
            ]
cols_out += new_cols
cols_out += ['Previous Tags', 'Needs Review', 'D4G Notes', 'CCMF Notes', 'Status']

rr_clean = rr_clean[cols_out]
print(rr_clean.shape)
rr_clean.head(2)

(1000, 29)


,Incident ID,Article URL,Incident Description,Publication Date,Incident Date,Pub Date Minus Incident Date,Province,City or Region,Detailed Location,Incident Category,...,South Asian,Southeast Asian,Identity-Based,Context,Type of Incident,Previous Tags,Needs Review,D4G Notes,CCMF Notes,Status
0,169,https://www.cbc.ca/news/canada/british-columbi...,RCMP are looking for a group of fishermen accu...,2018-08-29,2018-08-29,0.0,British Columbia,Hope,NaN,Criminal: Unknown,...,NaN,NaN,NaN,NaN,NaN,C; race relations; anti-indigenous; Fraser Riv...,Check city,"Location originally listed as Fraser River, wh...",NaN,NaN
1,831,https://www.theglobeandmail.com/politics/artic...,"Conservative leadership candidate, who was dis...",2020-03-24,2020-03-24,0.0,Ontario,Cambridge,NaN,Non-Criminal,...,NaN,NaN,Religious discrimination: Islamophobia,Politics,NaN,NC; political; anti-muslim; Cambridge,Check city,No mention of the city of Cambridge in the art...,NaN,NaN


In [9]:
rr_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Incident ID                   1000 non-null   int64         
 1   Article URL                   1000 non-null   object        
 2   Incident Description          999 non-null    object        
 3   Publication Date              823 non-null    datetime64[ns]
 4   Incident Date                 997 non-null    datetime64[ns]
 5   Pub Date Minus Incident Date  820 non-null    float64       
 6   Province                      941 non-null    object        
 7   City or Region                852 non-null    object        
 8   Detailed Location             26 non-null     object        
 9   Incident Category             734 non-null    object        
 10  Gender of Victim(s)           0 non-null      float64       
 11  Name of Victim(s)             0

In [10]:
cols = ['Incident Category', 'Gender of Victim(s)'] + new_cols
for col in cols:
    print('---- ' + col + ' ------')
    print(rr_clean[col].value_counts(dropna=False))
    print('')

---- Incident Category ------
Non-Criminal             640
NaN                      266
Criminal: Unknown         90
Criminal: Not charged      2
Criminal: Charged          2
Name: Incident Category, dtype: int64

---- Gender of Victim(s) ------
NaN    1000
Name: Gender of Victim(s), dtype: int64

---- Ethnic Community ------
NaN                                                    528
Black/African/Caribbean                                207
Indigenous                                             177
East Asian                                              27
South Asian                                             22
Middle Eastern                                           8
Black/African/Caribbean, Indigenous                      7
Black/African/Caribbean, Indigenous, Middle Eastern      7
Central Asian                                            5
Southeast Asian                                          4
Black/African/Caribbean, South Asian                     3
Latin American         

## Expand Columns

Create an expanded version of the dataset, to facilitate the manual revision process, where comma-separated values are split into separate columns, and with separate year, month, day columns for the incident date.

In [11]:
def split(series, sep=', '):
    """Split a series of comma-separated values into separate columns"""
    name = series.name
    expanded = series.str.split(sep, expand=True).replace({None: np.nan})
    columns = expanded.columns
    expanded.columns = [f'{name} {n + 1}' for n in columns]
    
    return expanded

In [12]:
rr_expand = rr_clean.copy()

# Column to flag approximate incident dates
rr_expand.insert(5, 'Month Only', [np.nan] * rr_clean.shape[0])

# Add a second gender column in case more than one victim
rr_expand = rr_expand.rename({'Gender of Victim(s)' : 'Gender of Victim(s) 1'}, axis=1)
rr_expand.insert(12, 'Gender of Victim(s) 2', [np.nan] * rr_clean.shape[0])

# Split comma-separated values into separate columns
cols_split = ['Ethnic Community', 'Identity-Based', 'Context', 'Type of Incident']
rr_expand = rr_expand.drop(cols_split, axis=1)
for col in cols_split:
    rr_expand = rr_expand.join(split(rr_clean[col]))
    
# Re-order columns
cols = rr_expand.columns.to_list()
cols_out = cols[:14] + cols[27:30] + cols[14:22] + cols[30:] + cols[22:27]
rr_expand = rr_expand[cols_out]

rr_expand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Incident ID                   1000 non-null   int64         
 1   Article URL                   1000 non-null   object        
 2   Incident Description          999 non-null    object        
 3   Publication Date              823 non-null    datetime64[ns]
 4   Incident Date                 997 non-null    datetime64[ns]
 5   Month Only                    0 non-null      float64       
 6   Pub Date Minus Incident Date  820 non-null    float64       
 7   Province                      941 non-null    object        
 8   City or Region                852 non-null    object        
 9   Detailed Location             26 non-null     object        
 10  Incident Category             734 non-null    object        
 11  Gender of Victim(s) 1         0

## Save to CSV

In [13]:
def save_data(data, savefile, index=False, float_format='%.0f'):
    print(f'Saving to {savefile}')
    data.to_csv(savefile, index=index, float_format=float_format)

In [14]:
if save_csv:
    save_data(rr_clean, 'race_relations_clean_prelim.csv')
    save_data(rr_expand, 'race_relations_clean_prelim_expanded_columns.csv')

Saving to race_relations_clean_prelim.csv
Saving to race_relations_clean_prelim_expanded_columns.csv
